<a href="https://colab.research.google.com/github/Akshay-ramagiri/AI/blob/main/19K41A0555_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import required libraries

In [74]:
import numpy as np
import math as mt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
import warnings
warnings.filterwarnings('ignore')

#Read the data

In [75]:
data = pd.read_csv('/content/drive/MyDrive/Assignment_5.csv')
data.head()

,Unnamed: 0,L(T-3),L(T-2),L(T-1),L(T)
0,1,5551.82208,4983.17184,4888.39680,5072.95872
1,2,5196.25980,5641.29720,7153.17732,7849.80504
2,3,6329.97504,6292.87560,5349.80160,5225.40936
3,4,5000.47452,4775.53968,6177.05676,6177.05676
4,5,6155.70120,6212.12976,7424.25264,7407.41760


##Normalization

In [76]:
def normalize(column):
  c_max, c_min = np.max(column), np.min(column)
  for i in range(len(column)):
    column.iloc[i] = (column.iloc[i] - c_min) / (c_max - c_min)
  return c_max, c_min

x1max, x1min = normalize(data['L(T-3)'])
x2max, x2min = normalize(data['L(T-2)'])
x3max, x3min = normalize(data['L(T-1)'])
ymax, ymin = normalize(data['L(T)'])
data.head()

,Unnamed: 0,L(T-3),L(T-2),L(T-1),L(T)
0,1,0.318003,0.119312,0.064412,0.121468
1,2,0.158090,0.497492,0.899988,1.000000
2,3,0.667975,0.871910,0.234645,0.169700
3,4,0.070036,0.000000,0.539855,0.470780
4,5,0.589596,0.825511,1.000000,0.860038


In [77]:
data.isnull().sum()

Unnamed: 0    0
L(T-3)        0
L(T-2)        0
L(T-1)        0
L(T)          0
dtype: int64

##Split data for testing and training

In [78]:
X = data[['L(T-3)', 'L(T-2)', 'L(T-1)']]
Y = data['L(T)']
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.1, random_state=101)

##Initialize Learning Rate, Epochs and Model Parameters

In [79]:
m = [np.random.uniform(-10, 10) for i in range(xtrain.shape[1])]
c = np.random.uniform(-10, 10)
eeta = 0.1
epochs = 500

##Train the model

In [80]:
for _ in range(epochs):
  for i in range(xtrain.shape[0]):
    y, ypred = ytrain.iloc[i], sum([xtrain.iat[i,z] * m[z] for z in range(xtrain.shape[1])])
    gradm = [-1 * xtrain.iat[i,j] * (y - ypred - c) for j in range(xtrain.shape[1])]
    gradc = -1 * (y - ypred - c)
    c = c - eeta * gradc
    for j in range(len(m)):
      m[j] = m[j] - eeta * gradm[j]

#Display Model Parameters after training
print(m, c) 

[-0.5178944933315242, 0.257678253892309, 0.9531022856720794] -0.007936362269933177


###Predict L(T) values for training and testing data

In [81]:
ytrainPred = []
for x in range(xtrain.shape[0]):
  ytrainPred.append(xtrain.iat[x, 0] * m[0] + xtrain.iat[x, 1] * m[1] + xtrain.iat[x, 2] * m[2] + c)
 
ytestPred = []
for x in range(xtest.shape[0]):
  ytestPred.append(xtest.iat[x, 0] * m[0] + xtest.iat[x, 1] * m[1] + xtest.iat[x, 2] * m[2] + c)

##Denormalize training, testing data and predicted values

In [82]:
xtrain['L(T-3)'] = xtrain['L(T-3)'] * (x1max - x1min) + x1min
xtrain['L(T-2)'] = xtrain['L(T-2)'] * (x2max - x2min) + x2min
xtrain['L(T-1)'] = xtrain['L(T-1)'] * (x3max - x3min) + x3min
ytrain = ytrain * (ymax - ymin) + ymin
xtest['L(T-3)'] = xtest['L(T-3)'] * (x1max - x1min) + x1min
xtest['L(T-2)'] = xtest['L(T-2)'] * (x2max - x2min) + x2min
xtest['L(T-1)'] = xtest['L(T-1)'] * (x3max - x3min) + x3min
ytest = ytest * (ymax - ymin) + ymin
ytrainPred = np.multiply(ytrainPred, (ymax - ymin)) + ymin
ytestPred = np.multiply(ytestPred, (ymax - ymin)) + ymin

##Error Metrics

In [83]:
MAE_Train = mean_absolute_error(ytrainPred, ytrain) #Testing MAE
MSE_Train = mean_squared_error(ytrainPred, ytrain) #Testing MSE
RMSE_Train = mean_squared_error(ytrainPred, ytrain) ** 0.5 #Testing RMSE
MAE_Test = mean_absolute_error(ytestPred, ytest) #Testing MAE
MSE_Test = mean_squared_error(ytestPred, ytest) #Testing MSE
RMSE_Test = mean_squared_error(ytestPred, ytest) ** 0.5 #Testing RMSE
print('Training Error :')
print('Mean Absolute Error :', MAE_Train)
print('Mean Squared Error :', MSE_Train)
print('Root Mean Squared Error :', RMSE_Train)
print('\nTesting Error :')
print('Mean Absolute Error :', MAE_Test)
print('Mean Squared Error :', MSE_Test)
print('Root Mean Squared Error :', RMSE_Test)

Training Error :
Mean Absolute Error : 167.86772024888393
Mean Squared Error : 44482.71806916852
Root Mean Squared Error : 210.9092650150024

Testing Error :
Mean Absolute Error : 638.3529267170379
Mean Squared Error : 407494.45904820797
Root Mean Squared Error : 638.3529267170379


##Analyze training data predictions

In [84]:
xtrain['Y_Train'] = ytrain
xtrain['Y_Train_Prediction'] = ytrainPred
xtrain

,L(T-3),L(T-2),L(T-1),Y_Train,Y_Train_Prediction
2,6329.97504,6292.87560,5349.80160,5225.40936,4987.515547
5,7068.22272,6515.78400,6350.08356,5502.25224,5660.105433
4,6155.70120,6212.12976,7424.25264,7407.41760,7383.695830
6,4931.26380,4775.53968,4713.81120,4689.02628,4600.248878
1,5196.25980,5641.29720,7153.17732,7849.80504,7521.600810
3,5000.47452,4775.53968,6177.05676,6177.05676,6175.632973
7,4844.75040,5571.46296,6150.71304,6296.30496,6633.504807


##Analyze testing data predictions

In [85]:
xtest['Y_Test'] = ytest
xtest['Y_Test_Prediction'] = ytestPred
xtest

,L(T-3),L(T-2),L(T-1),Y_Test,Y_Test_Prediction
0,5551.82208,4983.17184,4888.3968,5072.95872,4434.605793


##Deployment

In [86]:
2x1 = float(input('Enter load at T-3 hours : '))
x2 = float(input('Enter load at T-2 hours : '))
x3 = float(input('Enter load at T-1 hours : '))
x1Norm = (x1 - x1min) / (x1max - x1min)
x2Norm = (x2 - x2min) / (x2max - x2min)
x3Norm = (x3 - x3min) / (x3max - x3min)
yPred = x1Norm * m[0] + x2Norm * m[1] + x3Norm * m[2] + c
y = yPred * (ymax - ymin) + ymin
print('Load at T hour :', y)

Enter load at T-3 hours : 1500
Enter load at T-2 hours : 2000
Enter load at T-1 hours : 2500
Load at T hour : 3366.8300184738414
